Do a random forest return predictor for long time equity screens, maybe including rates/ sectors / commodities / geoploitic indicators. 
Find the % of regimes in the last 30 days that have a higher volatility than the current IV 
Add trend in shannon entropy or/and regime percentages to evaluate recessions / hostile environments beginning. could also look at earnings timestamps affecting options prices 

Want to find stocks with a low hv/iv high regime % in highest stdev state, and high # of days with stdev above avg iv on options 

In [1]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd
import time


In [2]:
from yfinance import EquityQuery, screen
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import time

# --- Helper Functions ---

def find_closest_value(data_list, target_value):
    if len(data_list) == 0:
        return None
    return min(data_list, key=lambda x: abs(x - target_value))

def filter_special_expiry(expiry_list):
    """Return the two soonest (earliest) expiry dates from the list."""
    sorted_expiries = sorted(expiry_list)
    return sorted_expiries[:2]

def get_prev_day_price_volume(symbol):
    """Fetch the most recent full day's close price and volume for the symbol."""
    try:
        data = yf.Ticker(symbol).history(period="5d")
        if data.shape[0] < 2:
            return None, None
        prev_day = data.iloc[-2]
        price = float(prev_day['Close'])
        volume = int(prev_day['Volume'])
        return price, volume
    except Exception as e:
        print(f"Error fetching previous day price/volume for {symbol}: {e}")
        return None, None

def get_historical_volatility(symbol, window=20):
    try:
        data = yf.Ticker(symbol).history(period=f"{window*2}d")
        if data.shape[0] < window + 1:
            return None
        data = data['Close'].dropna()
        log_returns = np.log(data / data.shift(1)).dropna()
        vol = log_returns[-window:].std() * np.sqrt(252)
        return vol
    except Exception as e:
        print(f"Error calculating historical volatility for {symbol}: {e}")
        return None

def get_options(symbol, max_retries=3, retry_delay=60):
    for attempt in range(max_retries):
        try:
            stock = yf.Ticker(symbol)
            options_chain = stock.options
            if not options_chain:
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
            filtered_expiries = filter_special_expiry(options_chain)
            options_data = []
            current_price, _ = get_prev_day_price_volume(symbol)
            if current_price is None:
                continue
            for expiry in filtered_expiries:
                chain = stock.option_chain(expiry)
                for opt_type, df in [('call', chain.calls), ('put', chain.puts)]:
                    if not df.empty:
                        closest_strike = find_closest_value(df['strike'], current_price)
                        filtered = df[df['strike'] == closest_strike].copy()
                        filtered['type'] = opt_type
                        filtered['expiry'] = expiry
                        filtered['symbol'] = symbol
                        options_data.append(filtered)
            if options_data:
                result = pd.concat(options_data, ignore_index=True)
                result = result[result['openInterest'] > 1000]
                return result[['strike', 'impliedVolatility', 'openInterest', 'symbol']]
            else:
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
        except Exception as e:
            print(f"Error fetching options for {symbol}: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
                continue
            else:
                print(f"Max retries reached for {symbol}.")
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
    return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])

# --- Screen Query ---
screen_query = EquityQuery("and", [
    EquityQuery("is-in", ["region", "us"]),
    EquityQuery("is-in", ["sector",
        "Technology",
        "Financial Services",
        "Consumer Cyclical",
        "Communication Services",
        "Basic Materials",
        "Industrials"
    ]),
    EquityQuery("GTE", ["eodprice", 0.50]),
    EquityQuery("GTE", ["avgdailyvol3m", 100000]),
    EquityQuery("LTE", ["short_percentage_of_float.value", 100])
])

# --- Batch Processing Function ---
def process_batch(offset, filterdf, options):
    batch_size = 250
    screener = screen(
        screen_query,
        sortField="avgdailyvol3m",
        sortAsc=False,
        size=batch_size,
        offset=offset
    )
    stocks = screener.get('quotes', [])
    if not stocks:
        print("No more stocks to load.")
        return filterdf, options, offset

    current_filterdf = pd.DataFrame(stocks)

    # --- Add price*volume filter for most recent full day ---
    current_filterdf['prev_day_price'] = None
    current_filterdf['prev_day_volume'] = None
    current_filterdf['price_volume'] = None

    for idx, row in current_filterdf.iterrows():
        symbol = row['symbol']
        price, volume = get_prev_day_price_volume(symbol)
        current_filterdf.at[idx, 'prev_day_price'] = price
        current_filterdf.at[idx, 'prev_day_volume'] = volume
        if price is not None and volume is not None:
            current_filterdf.at[idx, 'price_volume'] = price * volume

    current_filterdf = current_filterdf[current_filterdf['price_volume'] > 400_000_000]

    if current_filterdf.empty:
        print("No stocks passed the price*volume filter in this batch.")
        offset += batch_size
        return filterdf, options, offset

    current_filterdf['historical_volatility'] = None
    for idx, row in current_filterdf.iterrows():
        symbol = row['symbol']
        hv = get_historical_volatility(symbol, window=20)
        current_filterdf.at[idx, 'historical_volatility'] = hv

    filterstocks = current_filterdf['symbol'].tolist()
    options_list = []
    for symbol in filterstocks:
        options_data = get_options(symbol)
        options_list.append(options_data)

    if options_list:
        current_options = pd.concat(options_list, ignore_index=True)
        options = pd.concat([options, current_options], ignore_index=True)
    else:
        current_options = pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])

    if not current_options.empty:
        qualifying_symbols = set(current_options['symbol'].str.upper().unique())
        current_filterdf = current_filterdf[current_filterdf['symbol'].str.upper().isin(qualifying_symbols)]
    else:
        current_filterdf = current_filterdf.iloc[0:0]

    if current_filterdf.empty:
        print("No stocks passed the open interest filter in this batch.")
        offset += batch_size
        return filterdf, options, offset

    if not options.empty and 'symbol' in options.columns and 'impliedVolatility' in options.columns:
        avg_iv = options.groupby('symbol')['impliedVolatility'].mean().reset_index()
        avg_iv.columns = ['symbol', 'avg_iv']
    else:
        avg_iv = pd.DataFrame(columns=['symbol', 'avg_iv'])

    current_filterdf['symbol'] = current_filterdf['symbol'].astype(str).str.upper()
    avg_iv['symbol'] = avg_iv['symbol'].astype(str).str.upper()
    current_filterdf = current_filterdf.merge(avg_iv, on='symbol', how='left')

    if 'avg_iv' in current_filterdf.columns and 'historical_volatility' in current_filterdf.columns:
        current_filterdf["iv/hv_ratio"] = current_filterdf["avg_iv"] / current_filterdf["historical_volatility"]
    else:
        current_filterdf["iv/hv_ratio"] = None

    filterdf = pd.concat([filterdf, current_filterdf], ignore_index=True)

    offset += batch_size
    return filterdf, options, offset

# --- Main loop ---
offset = 0
filterdf = pd.DataFrame()
options = pd.DataFrame()

filterdf, options, offset = process_batch(offset, filterdf, options)
print("Initial batch:")
print(filterdf)

while True:
    user_input = input("Process next 100 stocks? (y/n): ")
    if user_input.lower() != 'y':
        break
    filterdf, options, offset = process_batch(offset, filterdf, options)
    print("Updated results:")
    print(filterdf)


Initial batch:
   language region quoteType typeDisp         quoteSourceName  triggerable  \
0     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
1     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
2     en-US     US    EQUITY   Equity           Delayed Quote        False   
3     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
4     en-US     US    EQUITY   Equity           Delayed Quote         True   
..      ...    ...       ...      ...                     ...          ...   
68    en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
69    en-US     US    EQUITY   Equity           Delayed Quote        False   
70    en-US     US    EQUITY   Equity           Delayed Quote        False   
71    en-US     US    EQUITY   Equity           Delayed Quote        False   
72    en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   

   customPriceAlertConfidence currency  hasPrePo

In [3]:
import yfinance as yf
import numpy as np
from hmmlearn.hmm import GaussianHMM
from tqdm import tqdm

def get_last_120_returns(symbol):
    """Get last 120 days of daily log returns for a symbol."""
    try:
        data = yf.Ticker(symbol).history(period="130d")
        closes = data['Close'].dropna()
        if len(closes) < 121:
            return None
        returns = np.log(closes / closes.shift(1)).dropna()
        return returns[-120:]
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def fit_hmm_and_stats(returns, n_states=3):
    """Fit HMM to returns, return means, stds, and state sequence."""
    model = GaussianHMM(
        n_components=n_states,
        covariance_type="diag",
        n_iter=500,
        tol=1e-2,
        random_state=42,
        verbose=False
    )
    X = returns.values.reshape(-1, 1)
    model.fit(X)
    hidden_states = model.predict(X)
    means = model.means_.flatten()
    stds = np.sqrt(np.array([np.diag(cov)[0] for cov in model.covars_]))
    return hidden_states, means, stds

def hmm_analysis(symbol):
    returns = get_last_120_returns(symbol)
    if returns is None:
        return {
            "hmm_mean_0": None,
            "hmm_std_0": None,
            "hmm_mean_1": None,
            "hmm_std_1": None,
            "hmm_mean_2": None,
            "hmm_std_2": None,
            "pct_last30_in_highest_stdev_state": None,
            "highest_stdev_state": None
        }
    hidden_states, means, stds = fit_hmm_and_stats(returns, n_states=3)
    # Identify the state with the highest stdev
    high_stdev_state = int(np.argmax(stds))
    last30 = hidden_states[-30:]
    pct_high = np.mean(last30 == high_stdev_state) * 100
    return {
        "hmm_mean_0": means[0],
        "hmm_std_0": stds[0],
        "hmm_mean_1": means[1],
        "hmm_std_1": stds[1],
        "hmm_mean_2": means[2],
        "hmm_std_2": stds[2],
        "pct_last30_in_highest_stdev_state": pct_high,
        "highest_stdev_state": high_stdev_state
    }

# --- Apply to filterdf ---

filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()

for col in [
    'hmm_mean_0','hmm_std_0',
    'hmm_mean_1','hmm_std_1',
    'hmm_mean_2','hmm_std_2',
    'pct_last30_in_highest_stdev_state',
    'highest_stdev_state'
]:
    filterdf[col] = None

for idx, row in tqdm(filterdf.iterrows(), total=len(filterdf)):
    symbol = row['symbol']
    hmm_stats = hmm_analysis(symbol)
    for col in hmm_stats:
        filterdf.at[idx, col] = hmm_stats[col]

# filterdf now has the HMM columns for states 0, 1, 2, and the % of last 30 closes in the highest stdev state.


100%|██████████| 73/73 [00:26<00:00,  2.75it/s]


In [4]:
N = 252  # Or use 261, depending on your convention

def pct_regimes_higher_than_iv(hidden_states, stds, avg_iv):
    """% of regimes in last 30 days with annualized stdev higher than avg_iv."""
    last30 = hidden_states[-30:]
    # Annualize regime stdevs
    stds_annualized = stds * np.sqrt(N)
    last30_vols_annualized = stds_annualized[last30]
    pct_higher = np.mean(last30_vols_annualized > avg_iv) * 100
    return pct_higher

filterdf['pct_regimes_last30_higher_than_avg_iv'] = None

for idx, row in tqdm(filterdf.iterrows(), total=len(filterdf)):
    symbol = row['symbol']
    avg_iv = row['avg_iv']
    returns = get_last_120_returns(symbol)
    if returns is None or avg_iv is None:
        filterdf.at[idx, 'pct_regimes_last30_higher_than_avg_iv'] = None
        continue
    hidden_states, means, stds = fit_hmm_and_stats(returns, n_states=3)
    pct_higher = pct_regimes_higher_than_iv(hidden_states, stds, avg_iv)
    filterdf.at[idx, 'pct_regimes_last30_higher_than_avg_iv'] = pct_higher


100%|██████████| 73/73 [00:22<00:00,  3.26it/s]


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import entropy
from hmmlearn.hmm import GaussianHMM
from statsmodels.tsa.stattools import adfuller

def shannon_entropy(series, bins=10):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

def hmm_regime_switch_prob_and_count(returns, n_states=2):
    returns = returns.values.reshape(-1, 1)
    model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=100)
    model.fit(returns)
    hidden_states = model.predict(returns)
    last_state = hidden_states[-1]
    transmat = model.transmat_
    switch_prob = 1 - transmat[last_state, last_state]
    regime_switch_count = np.sum(hidden_states[1:] != hidden_states[:-1])
    return switch_prob, regime_switch_count

entropy_list = []
regimeswitchprob_list = []
regimeswitchcount_list = []
adf_pvalue_list = []
avg_return_list = []

for ticker in filterdf['symbol']:
    try:
        # Download last 7 days of 1-minute data
        df = yf.download(ticker, period="7d", interval="1m", progress=False)
        closes = df['Close'].dropna()
        # Make sure you have enough data (e.g., at least 1000 minutes)
        if len(closes) < 1000:
            entropy_list.append(np.nan)
            regimeswitchprob_list.append(np.nan)
            regimeswitchcount_list.append(np.nan)
            adf_pvalue_list.append(np.nan)
            avg_return_list.append(np.nan)
            continue
        returns = np.log(closes).diff().dropna()
        # Shannon entropy
        ent = shannon_entropy(returns)
        entropy_list.append(ent)
        # HMM regime switch probability and count
        switch_prob, switch_count = hmm_regime_switch_prob_and_count(returns)
        regimeswitchprob_list.append(round(switch_prob * 100, 2))  # as percentage
        regimeswitchcount_list.append(int(switch_count))
        # ADF test p-value on last 7 days of minute returns
        adf_pvalue = adfuller(returns)[1]
        adf_pvalue_list.append(adf_pvalue)
        # Average return over last 7 days (mean of returns)
        avg_return = returns.mean()
        avg_return_list.append(avg_return)
    except Exception as e:
        entropy_list.append(np.nan)
        regimeswitchprob_list.append(np.nan)
        regimeswitchcount_list.append(np.nan)
        adf_pvalue_list.append(np.nan)
        avg_return_list.append(np.nan)

filterdf['ShannonEntropy'] = entropy_list
filterdf['Regimeswitchprob'] = regimeswitchprob_list
filterdf['Regimeswitchcount'] = regimeswitchcount_list
filterdf['ADF_Pvalue'] = adf_pvalue_list
filterdf['AvgReturn_7d'] = avg_return_list


Model is not converging.  Current: 13088.000527985547 is not greater than 13089.812175167084. Delta is -1.8116471815374098
Model is not converging.  Current: 12939.06560497801 is not greater than 12939.928035259714. Delta is -0.8624302817042917
Model is not converging.  Current: 12675.925292368553 is not greater than 12681.289433025113. Delta is -5.3641406565602665
Model is not converging.  Current: 12706.224798130828 is not greater than 12711.447130120101. Delta is -5.222331989272789
Model is not converging.  Current: 12233.21117681298 is not greater than 12240.704748601549. Delta is -7.49357178856917
Model is not converging.  Current: 12593.136008781401 is not greater than 12595.302971779936. Delta is -2.166962998535382
Model is not converging.  Current: 13155.740181661477 is not greater than 13156.012174231522. Delta is -0.27199257004576793
Model is not converging.  Current: 12501.251509979611 is not greater than 12501.931152648935. Delta is -0.6796426693235844
Model is not convergi

In [9]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
filterdf["Shannon/IV"] = filterdf["ShannonEntropy"] / filterdf["avg_iv"]
display(filterdf)


,language,region,quoteType,typeDisp,quoteSourceName,triggerable,customPriceAlertConfidence,currency,hasPrePostMarketData,firstTradeDateMilliseconds,priceHint,postMarketChangePercent,postMarketTime,postMarketPrice,postMarketChange,regularMarketChange,regularMarketTime,regularMarketPrice,regularMarketDayHigh,regularMarketDayRange,regularMarketDayLow,regularMarketVolume,regularMarketPreviousClose,bidSize,askSize,market,messageBoardId,fullExchangeName,longName,financialCurrency,regularMarketOpen,averageDailyVolume3Month,averageDailyVolume10Day,corporateActions,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekChangePercent,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,earningsCallTimestampStart,earningsCallTimestampEnd,isEarningsDateEstimate,trailingAnnualDividendRate,trailingPE,dividendRate,trailingAnnualDividendYield,marketState,epsTrailingTwelveMonths,epsForward,epsCurrentYear,priceEpsCurrentYear,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,sourceInterval,exchangeDataDelayedBy,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,esgPopulated,tradeable,cryptoTradeable,regularMarketChangePercent,exchange,bid,ask,fiftyTwoWeekHigh,fiftyTwoWeekLow,averageAnalystRating,dividendYield,shortName,displayName,symbol,ipoExpectedDate,prev_day_price,prev_day_volume,price_volume,historical_volatility,avg_iv,iv/hv_ratio,hmm_mean_0,hmm_std_0,hmm_mean_1,hmm_std_1,hmm_mean_2,hmm_std_2,pct_last30_in_highest_stdev_state,highest_stdev_state,pct_regimes_last30_higher_than_avg_iv,ShannonEntropy,Regimeswitchprob,Regimeswitchcount,ADF_Pvalue,Shannon/IV
0,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,True,917015400000,2,0.009095,1752278396,164.9350,0.014999,0.819992,1752264000,164.92,167.8900,163.52 - 167.89,163.5200,190872330,164.1000,1,1,us_market,finmb_32307,NasdaqGS,NVIDIA Corporation,USD,163.715,212667667,180881070,[],78.299995,0.903948,86.62 - 167.89,-2.970001,-0.017690,28.402365,1.751501e+09,1.756325e+09,1756324800,1756324800,1.748466e+09,1.748466e+09,False,0.040,53.200000,0.04,0.000244,CLOSED,3.10,4.12,4.28706,38.469257,24387600384,3.438,139.5280,25.391998,0.181985,131.179350,33.740646,0.257210,4022003040256,40.029125,47.969750,15,0,America/New_York,EDT,-14400000,False,False,False,0.499690,NMS,158.89,175.00,167.890,86.620,1.5 - Strong Buy,0.03,NVIDIA Corporation,NVIDIA,NVDA,NaN,164.100006,167704100,27520243833.584595,0.265497,0.319617,1.203846,-0.002046,0.034672,0.009143,0.025112,-0.019633,0.105408,0.0,2,100.0,1.154467,0.07,2,0.000000e+00,3.612029
1,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,True,1277818200000,2,-0.226475,1752278399,312.8000,-0.710022,3.640010,1752264000,313.51,314.0900,305.66 - 314.09,305.6600,77499572,309.8700,1,1,us_market,finmb_27444752,NasdaqGS,"Tesla, Inc.",USD,307.620,112076406,98167240,[],131.510010,0.722582,182.0 - 488.54,-175.030000,-0.358272,24.093569,NaN,1.753301e+09,1753300800,1753300800,1.753306e+09,1.753306e+09,False,0.000,178.130690,NaN,0.000000,CLOSED,1.76,3.24,1.86876,167.763660,3220960000,23.184,320.0774,-6.567383,-0.020518,316.233760,-2.723755,-0.008613,1009803198464,96.762350,13.522689,15,0,America/New_York,EDT,-14400000,False,False,False,1.174690,NMS,313.31,313.53,488.540,182.000,2.6 - Hold,NaN,"Tesla, Inc.",Tesla,TSLA,NaN,309.869995,104365300,32339675001.403809,0.568352,0.532842,0.93752,-0.003772,0.115664,-0.003889,0.065026,-0.002003,0.037615,3.333333,0,100.0,0.212268,0.12,6,2.455137e-30,0.398370
2,en-US,US,EQUITY,Equity,Delayed Quote,False,LOW,USD,True,76253400000,2,0.000000,1752278397,11.7800,0.000000,-0.130000,1752264002,11.78,11.8600,11.71 - 11.86,11.7100,61473633,11.9100,510,555,us_market,finmb_106335,NYSE,Ford Motor Company,USD,11.850,101326221,93778660,[],3.340000,0.395735,8.44 - 14

In [7]:
import datetime

today = datetime.datetime.today().strftime('%Y-%m-%d')
filename = f'/Users/nshaffer/Desktop/equity_vol_screen_{today}.csv'
filterdf.to_csv(filename, index=False)
print(f"Results saved to {filename}")


Results saved to /Users/nshaffer/Desktop/equity_vol_screen_2025-07-12.csv
